In [ ]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

from src.kinematics import add_kinematics

## Data Preparation

In [76]:
input_dfs = []
output_dfs = []
for i in range(1, 2):
    input_dfs.append(pd.read_csv(f"data/train/input_2023_w{i:02}.csv"))
    output_dfs.append(pd.read_csv(f"data/train/output_2023_w{i:02}.csv"))
suppl_data = pd.read_csv("data/supplementary_data.csv")

input_df = pd.concat(input_dfs, ignore_index=True)
output_df = pd.concat(output_dfs, ignore_index=True)


/var/folders/_m/rvnpg_cs6xzcz0vlml0lzkbm0000gp/T/ipykernel_94920/1363257619.py:6: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  suppl_data = pd.read_csv("data/supplementary_data.csv")


In [80]:
  input_dfs = []
  output_dfs = []
  for i in range(1, 3):
      input_dfs.append(pd.read_csv(f"data/train/input_2023_w{i:02}.csv"))
      output_dfs.append(pd.read_csv(f"data/train/output_2023_w{i:02}.csv"))

  suppl_data = pd.read_csv("data/supplementary_data.csv")

  input_df = pd.concat(input_dfs, ignore_index=True)
  output_df = pd.concat(output_dfs, ignore_index=True)


/var/folders/_m/rvnpg_cs6xzcz0vlml0lzkbm0000gp/T/ipykernel_94920/2367233655.py:7: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  suppl_data = pd.read_csv("data/supplementary_data.csv")


In [81]:
input_df.head(5)

,game_id,play_id,player_to_predict,nfl_id,frame_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,player_birth_date,player_position,player_side,player_role,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y
0,2023090700,101,False,54527,1,right,42,Bryan Cook,6-1,210,1999-09-07,FS,Defense,Defensive Coverage,52.33,36.94,0.09,0.39,322.40,238.24,21,63.259998,-0.22
1,2023090700,101,False,54527,2,right,42,Bryan Cook,6-1,210,1999-09-07,FS,Defense,Defensive Coverage,52.33,36.94,0.04,0.61,200.89,236.05,21,63.259998,-0.22
2,2023090700,101,False,54527,3,right,42,Bryan Cook,6-1,210,1999-09-07,FS,Defense,Defensive Coverage,52.33,36.93,0.12,0.73,147.55,240.60,21,63.259998,-0.22
3,2023090700,101,False,54527,4,right,42,Bryan Cook,6-1,210,1999-09-07,FS,Defense,Defensive Coverage,52.35,36.92,0.23,0.81,131.40,244.25,21,63.259998,-0.22
4,2023090700,101,False,54527,5,right,42,Bryan Cook,6-1,210,1999-09-07,FS,Defense,Defensive Coverage,52.37,36.90,0.35,0.82,123.26,244.25,21,63.259998,-0.22


In [82]:
input_df[['game_id','play_id']].drop_duplicates().shape

(1669, 2)

In [48]:
output_df.head()

,game_id,play_id,nfl_id,frame_id,x,y
0,2023090700,101,46137,1,56.22,17.28
1,2023090700,101,46137,2,56.63,16.88
2,2023090700,101,46137,3,57.06,16.46
3,2023090700,101,46137,4,57.48,16.02
4,2023090700,101,46137,5,57.91,15.56


In [14]:
# Sample n unique (game_id, play_id) combinations
n_plays = 500
sampled_plays = input_df[['game_id', 'play_id']].drop_duplicates().sample(n=n_plays)

# Keep all rows with those (game_id, play_id) combinations
test = input_df.merge(sampled_plays, on=['game_id', 'play_id'])

In [24]:
kinematics_test = add_kinematics(test, smooth_window=1)
test_with_kinematics = test.merge(kinematics_test, on=['game_id', 'play_id', 'nfl_id', "frame_id"], suffixes = ('', '_kinematics'))

/Users/kniu91/Documents/projects/bdb-26/kinematics.py:126: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  out = df.groupby(keys, group_keys=False).apply(per_player)


## From output_df, create dir, s, and a for all players

In [26]:
correlations = test_with_kinematics[['s', 's_kinematics', 'a', 'a_kinematics','a_tan', 'dir', 'dir_kinematics']].corr()
correlations

,s,s_kinematics,a,a_kinematics,a_tan,dir,dir_kinematics
s,1.000000,0.997720,0.206025,0.141037,-0.055807,-0.032887,-0.009377
s_kinematics,0.997720,1.000000,0.205475,0.155265,-0.035968,-0.034154,-0.010450
a,0.206025,0.205475,1.000000,0.818378,0.355705,-0.007756,0.013478
a_kinematics,0.141037,0.155265,0.818378,1.000000,0.402249,-0.014473,0.003552
a_tan,-0.055807,-0.035968,0.355705,0.402249,1.000000,0.000224,0.006937
dir,-0.032887,-0.034154,-0.007756,-0.014473,0.000224,1.000000,0.934635
dir_kinematics,-0.009377,-0.010450,0.013478,0.003552,0.006937,0.934635,1.000000


## Catch Time Prediction

In [72]:
input_df.loc[
  (input_df['frame_id'] == 1)
  &
  (input_df['play_direction'] == 'right')
  & 
  (input_df['absolute_yardline_number']  > 105)
  &
  (input_df['player_position'] == "QB"),
  ['play_direction', 'x', 'y', 'absolute_yardline_number', 'x_std','y_std','absolute_yardline_number_std']
  ]

,play_direction,x,y,absolute_yardline_number,x_std,y_std,absolute_yardline_number_std
4446,right,104.71,23.60,109,-4.29,23.60,109
8568,right,101.68,29.92,106,-4.32,29.92,106
26403,right,100.86,23.77,106,-5.14,23.77,106
84937,right,103.63,23.91,109,-5.37,23.91,109
85505,right,104.23,23.88,109,-4.77,23.88,109
103032,right,101.09,28.63,106,-4.91,28.63,106
172743,right,104.22,29.72,106,-1.78,29.72,106
202302,right,108.21,23.50,109,-0.79,23.50,109
216908,right,101.07,23.80,106,-4.93,23.80,106


In [73]:
input_df.loc[
  (input_df['frame_id'] == 1)
  &
  (input_df['play_direction'] == 'left')
  & 
  (input_df['absolute_yardline_number']  < 15)
  &
  (input_df['player_position'] == "QB"),
  ['play_direction', 'x', 'y', 'absolute_yardline_number', 'x_std','y_std','absolute_yardline_number_std']
  ]

,play_direction,x,y,absolute_yardline_number,x_std,y_std,absolute_yardline_number_std
65061,left,14.46,23.50,13,-1.46,29.80,107
134024,left,17.98,23.78,13,-4.98,29.52,107
179448,left,17.09,23.61,13,-4.09,29.69,107
205992,left,12.89,29.48,11,-1.89,23.82,109
254877,left,11.70,29.80,11,-0.70,23.50,109
283809,left,15.13,24.06,13,-2.13,29.24,107


In [74]:
input_df['player_role'].value_counts()

player_role
Defensive Coverage    155397
Other Route Runner     84063
Targeted Receiver      23151
Passer                 23103
Name: count, dtype: int64

In [ ]:
# Normalize field direction and other stuff
# Flip so all plays go left to right
# Shift all plays so LOS is at x = 0
# Justification: absollute_yardline_number is NOT from perspective of offense
# Rather it is absolute position on field, and always increases from left to right
# All x values are relative to scale of 0-120 yards, with middle of field at 60 yards``

input_df[input_df['play_direction'] == 'left']
L = 120  # field length in yards
W = 53.3  # field width in yards

input_df['absolute_yardline_number_std'] = np.where(
    input_df['play_direction'] == 'left',
    L - input_df['absolute_yardline_number'],
    input_df['absolute_yardline_number'])

input_df["x_std"] = np.where(
    input_df['play_direction'] == 'left',
    L - input_df['x'],
    input_df['x']) - input_df['absolute_yardline_number_std']

input_df["y_std"] = np.where(
    input_df['play_direction'] == 'left',
    W - input_df['y'],
    input_df['y'])

input_df["ball_land_x_std"] = np.where(
    input_df['play_direction'] == 'left',
    L - input_df['ball_land_x'],
    input_df['ball_land_x']) - input_df['absolute_yardline_number_std']

input_df["ball_land_y_std"] = np.where(
    input_df['play_direction'] == 'left',
    W - input_df['ball_land_y'],
    input_df['ball_land_y'])

input_df["o_std"] = np.where(
    input_df['play_direction'] == 'left',
    (input_df['o'] + 180) % 360,
    input_df['o']
)

input_df["dir_std"] = np.where(
    input_df['play_direction'] == 'left',
    (input_df['dir'] + 180) % 360,
    input_df['dir']
)

In [ ]:
# Fields we can fill down
absolute_yardline_number_std
ball_land_x_std
ball_land_y_std
passer_x_y_at_time_of_throw
passer_speed_at_time_of_throw
passer_dir_vs_ball_dir_at_time_of_throw

In [44]:
# Apply kinematics to output dataset
output_df = add_kinematics(output_df, smooth_window=3)

/Users/kniu91/Documents/projects/bdb-26/kinematics.py:126: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  out = df.groupby(keys, group_keys=False).apply(per_player)


In [45]:
output_df.head()

,game_id,play_id,nfl_id,frame_id,x,y,vx,vy,s,s_mph,a,a_tan,a_mag,dir
0,2023090700,101,44930,1,53.20,13.98,7.550000,-2.100000,7.836613,16.025873,2.673537,-0.674805,2.311280,105.543679
1,2023090700,101,44930,2,53.96,13.78,7.450000,-2.250000,7.782352,15.914909,3.207743,-0.840558,3.035795,106.805021
2,2023090700,101,44930,3,54.70,13.54,7.233333,-2.566667,7.675213,15.695810,4.219549,-0.989110,4.276154,109.536655
3,2023090700,101,44930,4,55.41,13.27,6.933333,-2.983333,7.547939,15.435536,5.161936,-0.885828,5.346699,113.281652
4,2023090700,101,44930,5,56.09,12.95,6.616667,-3.483333,7.477559,15.291608,5.718227,-0.602123,5.862954,117.764488


In [34]:
input_df.head()

,game_id,play_id,player_to_predict,nfl_id,frame_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,player_birth_date,player_position,player_side,player_role,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y
0,2023090700,101,False,54527,1,right,42,Bryan Cook,6-1,210,1999-09-07,FS,Defense,Defensive Coverage,52.33,36.94,0.09,0.39,322.40,238.24,21,63.259998,-0.22
1,2023090700,101,False,54527,2,right,42,Bryan Cook,6-1,210,1999-09-07,FS,Defense,Defensive Coverage,52.33,36.94,0.04,0.61,200.89,236.05,21,63.259998,-0.22
2,2023090700,101,False,54527,3,right,42,Bryan Cook,6-1,210,1999-09-07,FS,Defense,Defensive Coverage,52.33,36.93,0.12,0.73,147.55,240.60,21,63.259998,-0.22
3,2023090700,101,False,54527,4,right,42,Bryan Cook,6-1,210,1999-09-07,FS,Defense,Defensive Coverage,52.35,36.92,0.23,0.81,131.40,244.25,21,63.259998,-0.22
4,2023090700,101,False,54527,5,right,42,Bryan Cook,6-1,210,1999-09-07,FS,Defense,Defensive Coverage,52.37,36.90,0.35,0.82,123.26,244.25,21,63.259998,-0.22


In [30]:
output_df[['game_id','play_id']].drop_duplicates()

,game_id,play_id
0,2023090700,101
63,2023090700,194
108,2023090700,219
132,2023090700,361
180,2023090700,436
...,...,...
31907,2023091100,3618
31962,2023091100,3684
31998,2023091100,3716
32019,2023091100,3942


In [38]:
idx = output_df.groupby(['game_id', 'play_id', 'nfl_id'])['frame_id'].idxmax()
# Use those indices to get the actual rows
max_frame_rows = output_df.loc[idx]

In [39]:
max_frame_rows

,game_id,play_id,nfl_id,frame_id,x,y
62,2023090700,101,44930,21,62.49,2.83
20,2023090700,101,46137,21,62.87,4.63
41,2023090700,101,52546,21,52.48,5.38
107,2023090700,194,41325,9,84.06,20.70
71,2023090700,194,44888,9,84.68,26.97
...,...,...,...,...,...,...
32036,2023091100,3942,47881,9,81.24,38.93
32045,2023091100,3942,54469,9,81.94,43.80
32065,2023091100,3987,41243,11,82.67,9.24
32087,2023091100,3987,42489,11,74.69,13.21


In [40]:
max_frame_rows[
  (max_frame_rows['game_id'] == 2023090700)
  & (max_frame_rows['play_id'] == 101)
  ]

,game_id,play_id,nfl_id,frame_id,x,y
62,2023090700,101,44930,21,62.49,2.83
20,2023090700,101,46137,21,62.87,4.63
41,2023090700,101,52546,21,52.48,5.38
